In [19]:
import pandas as pd
import numpy as np
import os
import math

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
# use get_db_url function to connect to the codeup db
from env import get_db_url
from sklearn.impute import SimpleImputer

# custom module imports
# import acquire as aq
import wrangle

In [2]:
df = wrangle.zillow_data()

In [3]:
df.head()

,typeconstructiontypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,bedroomcnt,...,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,typeconstructiondesc
0,NaN,NaN,NaN,NaN,NaN,14297519,1727539,NaN,3.5,4.0,...,NaN,6.059063e+13,0,0.025595,2017-01-01,None,None,None,None,None
1,NaN,NaN,NaN,NaN,NaN,17052889,1387261,NaN,1.0,2.0,...,NaN,6.111001e+13,1,0.055619,2017-01-01,None,None,None,None,None
2,NaN,NaN,NaN,NaN,NaN,14186244,11677,NaN,2.0,3.0,...,NaN,6.059022e+13,2,0.005383,2017-01-01,None,None,None,None,None
3,NaN,2.0,NaN,NaN,NaN,12177905,2288172,NaN,3.0,4.0,...,NaN,6.037300e+13,3,-0.103410,2017-01-01,None,None,None,Central,None
4,NaN,2.0,NaN,NaN,1.0,10887214,1970746,NaN,3.0,3.0,...,NaN,6.037124e+13,4,0.006940,2017-01-01,Central,None,None,Central,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        223 non-null    float64
 1   heatingorsystemtypeid         49571 non-null  float64
 2   buildingclasstypeid           15 non-null     float64
 3   architecturalstyletypeid      207 non-null    float64
 4   airconditioningtypeid         25007 non-null  float64
 5   parcelid                      77579 non-null  int64  
 6   id                            77579 non-null  int64  
 7   basementsqft                  50 non-null     float64
 8   bathroomcnt                   77579 non-null  float64
 9   bedroomcnt                    77579 non-null  float64
 10  buildingqualitytypeid         49809 non-null  float64
 11  calculatedbathnbr             76963 non-null  float64
 12  decktypeid                    614 non-null    float64
 13  f

In [5]:
for col in df.columns:
    if col != 'id':
        print(col)
        print(df[col].value_counts())

typeconstructiontypeid
6.0     220
13.0      1
10.0      1
4.0       1
Name: typeconstructiontypeid, dtype: int64
heatingorsystemtypeid
2.0     33633
7.0     14107
24.0      850
6.0       777
20.0      104
13.0       60
18.0       23
1.0        13
10.0        3
11.0        1
Name: heatingorsystemtypeid, dtype: int64
buildingclasstypeid
4.0    14
3.0     1
Name: buildingclasstypeid, dtype: int64
architecturalstyletypeid
7.0     173
8.0      19
21.0      7
2.0       5
3.0       3
Name: architecturalstyletypeid, dtype: int64
airconditioningtypeid
1.0     23186
13.0     1575
5.0       167
11.0       53
9.0        26
Name: airconditioningtypeid, dtype: int64
parcelid
11991059    3
10857130    3
12612211    3
11721753    2
14008322    2
           ..
11994786    1
14187514    1
11446758    1
14013599    1
12826780    1
Name: parcelid, Length: 77381, dtype: int64
basementsqft
900.0     2
700.0     2
100.0     2
640.0     2
515.0     2
273.0     2
912.0     2
314.0     1
819.0     1
1809.0    

In [6]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [7]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 70

In [8]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclasstypeid,77564,99.980665
buildingclassdesc,77564,99.980665
finishedsquarefeet13,77537,99.945862
basementsqft,77529,99.935550
storytypeid,77529,99.935550
yardbuildingsqft26,77509,99.909769
fireplaceflag,77407,99.778291
architecturalstyletypeid,77372,99.733175
architecturalstyledesc,77372,99.733175
typeconstructiontypeid,77356,99.712551


In [9]:
df.taxdelinquencyflag.value_counts()

Y    2900
Name: taxdelinquencyflag, dtype: int64

In [10]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['parcelid', 'num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [11]:
nulls_by_row(df)

,parcelid,num_cols_missing,percent_cols_missing
55652,14341728,47,70.149254
41670,167687839,47,70.149254
13457,167686999,47,70.149254
14547,167687739,46,68.656716
16644,167689317,46,68.656716
...,...,...,...
58524,14292517,23,34.328358
66285,14128839,23,34.328358
39389,14371475,23,34.328358
58338,14215623,22,32.835821


In [12]:
def summarize(df):
    '''
    This function will take in a single argument (a pandas dataframe) and 
    output to console various statistices on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # value_counts()
    # observation of nulls in the dataframe
    '''
    print('----------------------')
    print('Dataframe head')
    print(df.head(3))
    print('----------------------')
    print('Dataframe Info ')
    print(df.info())
    print('----------------------')
    print('Dataframe Description')
    print(df.describe())
    print('----------------------')
    num_cols = [col for col in df.columns if df[col].dtype != 'object']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('----------------------')
    print('Dataframe value counts ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts())
        else:
            # define bins for continuous columns and don't sort them
            print(df[col].value_counts(bins=10, sort=False))
    print('----------------------')
    print('nulls in df by column')
    print(nulls_by_col(df))
    print('----------------------')
    print('null in df by row')
    print(nulls_by_row(df))
    print('----------------------')


In [13]:
summarize(df)

----------------------
Dataframe head
   typeconstructiontypeid  heatingorsystemtypeid  buildingclasstypeid  \
0                     NaN                    NaN                  NaN   
1                     NaN                    NaN                  NaN   
2                     NaN                    NaN                  NaN   

   architecturalstyletypeid  airconditioningtypeid  parcelid       id  \
0                       NaN                    NaN  14297519  1727539   
1                       NaN                    NaN  17052889  1387261   
2                       NaN                    NaN  14186244    11677   

   basementsqft  bathroomcnt  bedroomcnt  buildingqualitytypeid  \
0           NaN          3.5         4.0                    NaN   
1           NaN          1.0         2.0                    NaN   
2           NaN          2.0         3.0                    NaN   

   calculatedbathnbr  decktypeid  finishedfloor1squarefeet  \
0                3.5         NaN             

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [ ]:
df.parcelid.value_counts()

11991059    3
10857130    3
12612211    3
11721753    2
14008322    2
           ..
11994786    1
14187514    1
11446758    1
14013599    1
12826780    1
Name: parcelid, Length: 77381, dtype: int64

In [ ]:
df.unitcnt.isnull().sum()

26876

In [ ]:
df.bedroomcnt.value_counts()

3.0     30436
2.0     19223
4.0     17551
5.0      4550
1.0      3391
6.0      1000
0.0       837
8.0       253
7.0       208
9.0        70
10.0       31
12.0       16
11.0        9
13.0        2
14.0        1
16.0        1
Name: bedroomcnt, dtype: int64

In [ ]:
def get_upper_outliers(s, k=1.5):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, 0.75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [ ]:
def add_upper_outlier_columns(df, k=1.5):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    for col in df.select_dtypes('float64'):
        df[col + '_outliers_upper'] = get_upper_outliers(df[col], k)
    return df

In [ ]:
df = add_upper_outlier_columns(df)

In [ ]:
outlier_cols = [col for col in df.columns if col.endswith('_outliers_upper')]
for col in outlier_cols:
    print(col, ': ')
    subset = df[col][df[col] > 0]
    print(f'Number of Observations Above Upper Bound: {subset.count()}', '\n')
    print(subset.describe())
    print('------', '\n')

typeconstructiontypeid_outliers_upper : 
Number of Observations Above Upper Bound: 2 

count    2.00000
mean     5.50000
std      2.12132
min      4.00000
25%      4.75000
50%      5.50000
75%      6.25000
max      7.00000
Name: typeconstructiontypeid_outliers_upper, dtype: float64
------ 

heatingorsystemtypeid_outliers_upper : 
Number of Observations Above Upper Bound: 977 

count    977.000000
mean       8.932958
std        1.493791
min        3.500000
25%        9.500000
50%        9.500000
75%        9.500000
max        9.500000
Name: heatingorsystemtypeid_outliers_upper, dtype: float64
------ 

buildingclasstypeid_outliers_upper : 
Number of Observations Above Upper Bound: 0 

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: buildingclasstypeid_outliers_upper, dtype: float64
------ 

architecturalstyletypeid_outliers_upper : 
Number of Observations Above Upper Bound: 26 

count    26.000000
mean      4.500000
std       

## Prepare
Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
A dataframe
A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
The output:
The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function

In [ ]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_row=0.8):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold) #1, or ‘columns’ : Drop columns which contain missing value
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold) #0, or ‘index’ : Drop rows which contain missing values.
    return df

In [ ]:
def data_prep(df, cols_to_remove=[], prop_required_column=0.5, prop_required_row=0.8):
    df = df.drop(columns=cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [ ]:
df = handle_missing_values(df)

In [ ]:
def trim_bad_data_zillow(df):
    # If it's not single unit, it's not a single family home.
    df = df[~(df.unitcnt > 1)]
    # If the lot size is smaller than the finished square feet, it's probably bad data or not a single family home
    df = df[~(df.lotsizesquarefeet < df.calculatedfinishedsquarefeet)]
    # If the finished square feet is less than 500 it is likeley an apartment, or bad data
    #df = df[~(df.calculatedfinishedsquarefeet < 500)]
    # If there are no bathrooms, it is not a home
    df = df[~(df.bathroomcnt < 1)]
    #If there are no bedrooms, it is not a home
    df = df[~(df.bedroomcnt < 1)]
    # remove duplicate parcelids
    df = df.sort_values('transactiondate').drop_duplicates('parcelid',keep='last')
    return df

In [ ]:
df = trim_bad_data_zillow(df)

In [11]:
df.transactiondate.dtypes

dtype('O')

In [13]:
df['transactiondate'] = pd.to_datetime(df['transactiondate'], format= '%Y/%m/%d')

In [14]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [15]:
df = add_date_features(df)

In [16]:
df.transaction_day.value_counts()

28    3747
31    3472
14    3041
30    2945
7     2936
24    2798
17    2794
1     2777
23    2622
21    2596
6     2561
12    2520
11    2516
10    2481
16    2468
15    2456
19    2442
26    2441
8     2432
25    2428
13    2397
9     2395
5     2394
18    2393
27    2321
22    2138
3     2063
20    2021
2     1845
29    1639
4     1500
Name: transaction_day, dtype: int64

In [17]:
df.transaction_quarter.value_counts()

2    30615
3    24213
1    22751
Name: transaction_quarter, dtype: int64

In [18]:
df.transaction_month.value_counts()

6    11453
5    10484
8     9930
7     9486
3     9324
4     8678
1     7019
2     6408
9     4797
Name: transaction_month, dtype: int64

In [23]:
df['longitude'].astype(float)

0       -117869207.0
1       -119281531.0
2       -117823170.0
3       -118240722.0
4       -118414640.0
            ...     
77574   -118502000.0
77575   -118282383.0
77576   -118706327.0
77577   -118038169.0
77578   -117996709.0
Name: longitude, Length: 77579, dtype: float64

In [24]:
df['latitude'].astype(float)

0        33634931.0
1        34449266.0
2        33886168.0
3        34245180.0
4        34185120.0
            ...    
77574    34202400.0
77575    34245368.0
77576    34300140.0
77577    34040895.0
77578    33937685.0
Name: latitude, Length: 77579, dtype: float64

In [28]:
def xy(df):
    df['x'] = math.cos(df.latitude) * math.cos(df.longitude)
    df['y'] = math.cos(df.latitude) * math.sin(df.longitude)
    return df

df.apply

TypeError: cannot convert the series to <class 'float'>

In [29]:
df['coslat'] = df.latitude.apply(math.cos)

In [30]:
df['coslong'] = df.longitude.apply(math.cos)

In [31]:
df['sinlong'] = df.longitude.apply(math.sin)

In [32]:
df['x'] = df.coslong * df.coslat

In [33]:
df['y'] = df.coslat * df.sinlong

In [ ]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)